In [26]:
import pandas as pd
import numpy as np
import sys
import pickle
from copy import deepcopy

from gale_shapley import Multi_Slot, Single_Slot, gale_shapley_algorithm

NUM_CHOICES = 12
rng = np.random.default_rng(seed=7)

YEAR = 2020
DISTRICT_FILE = f'./{YEAR}/starting_data/{YEAR}_Fall_district.csv'
SCHOOL_FILE = f'./{YEAR}/starting_data/{YEAR}_Fall_school.csv'
HAVE_RESULTS = False


In [36]:
student_numbers_df = pd.read_csv(DISTRICT_FILE)[['School District', 'Category', 'Grade 9 Offers']].replace(',','', regex=True)
special_schools = pd.read_csv(SCHOOL_FILE)[['School DBN', 'School District', 'Category', 'Grade 9 Offers']].replace(',','', regex=True)
special_high_schools = ['10X445', '13K430', '05M692', '10X696', '28Q687', '31R605', '02M475', '14K449']

special_schools = special_schools[special_schools['School DBN'].isin(special_high_schools)]
student_numbers_df = student_numbers_df[student_numbers_df['Category'].str.contains('Residential') | student_numbers_df['Category'].str.contains('All S')]
# student_numbers_df['Grade 9 Applicants'] = student_numbers_df['Grade 9 Applicants'].as_type('int')
#Now we count the total number of applicants from each district
totals = student_numbers_df[student_numbers_df['Category'] == 'All Students'].set_index('School District', drop=False)#.astype('int32')
totals

,School District,Category,Grade 9 Offers
School District,,,
1,1,All Students,1172
2,2,All Students,10323
3,3,All Students,2028
4,4,All Students,1309
5,5,All Students,1125
6,6,All Students,1536
7,7,All Students,1713
8,8,All Students,1801
9,9,All Students,2337


In [43]:
special_schools[special_schools['Category'] == 'All Students']['Grade 9 Offers'].astype('int32').sum()

4265

In [28]:
replace_on_row = lambda x : rng.integers(low=0, high=5, endpoint=True) if x == 's' else x

student_numbers_df['Grade 9 Offers'] = student_numbers_df['Grade 9 Offers'].apply(replace_on_row).astype('int32')
student_numbers_df[student_numbers_df['Category'] == 'All Students'].sum()
                    #['Grade 9 Applicants'].sum())


School District                                                  528
Category           All StudentsAll StudentsAll StudentsAll Studen...
Grade 9 Offers                                                 82140
dtype: object

In [10]:
non_s_totals = student_numbers_df[(~student_numbers_df['Grade 9 Applicants'].str.contains('s')) & student_numbers_df['Category'].str.contains('Residential')]#.groupby('School District').count())
non_s_totals = non_s_totals.astype({'Grade 9 Applicants':'int32'})
non_s_totals.groupby('School District').sum()

,Grade 9 Applicants
School District,
1,11072
2,36285
3,14174
4,8873
5,17876
6,5172
7,7381
8,5359
9,9016


In [2]:
non_s_totals = non_s_totals.groupby('School District').sum()
non_s_totals['Grade 9 Applicants'] = totals['Grade 9 Applicants'].astype('int32') - non_s_totals['Grade 9 Applicants'] #non_s_totals.join(totals, lsuffix='non_s')
print(non_s_totals)

                 School District      Category Grade 9 Applicants
School District                                                  
1                              1  All Students              11072
2                              2  All Students              36220
3                              3  All Students              14174
4                              4  All Students               8873
5                              5  All Students              17858
6                              6  All Students               5172
7                              7  All Students               7395
8                              8  All Students               5367
9                              9  All Students               9020
10                            10  All Students              23041
11                            11  All Students               8790
12                            12  All Students               5005
13                            13  All Students              23917
14        

<ipython-input-2-aef23ef23e2f>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  non_s_totals['Grade 9 Applicants'] = non_s_totals['Grade 9 Applicants'].astype('int32')
